In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%capture
!pip install datasets

In [5]:
from gensim.models import FastText
from nltk.tokenize import word_tokenize
from datasets import load_dataset
import nltk
import re
import pandas as pd
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [24]:
PATH = "/content/drive/MyDrive/instruction-tuning-mkn"

In [16]:
# tambahkan dataset
a = pd.read_csv(f"{PATH}/source/tapaleuk_new.csv")
b = pd.read_csv(f"{PATH}/source/jakarta_field_station_full_sentence.csv")['mkn_sentence_full']
c = pd.read_csv(f"{PATH}/source/tapaleuk_full_sentence.csv")['mkn_sentence_full']
d = load_dataset("cis-lmu/Taxi1500-RawData", name="mkn_Latn")['taxi1500']
e = pd.read_csv(f"{PATH}/source/puisi.csv", encoding='latin1')

In [18]:
# Prepare data for FastText
fasttext_data = []

# Extract from dataset a
fasttext_data.extend(a['text'].dropna().tolist())

# Extract from dataset b
fasttext_data.extend(b.dropna().tolist())

# Extract from dataset c
fasttext_data.extend(c.dropna().tolist())

# Extract from dataset d
fasttext_data.extend(d['text'])

# Extract from dataset e
fasttext_data.extend(e['text'])

In [25]:
len(fasttext_data)

18654

In [32]:
sentences = [word_tokenize(data.lower()) for data in fasttext_data]

In [33]:
fasttext_model = FastText(sentences=sentences, vector_size=300, window=5, min_count=2, workers=2)
w2v = fasttext_model.wv

In [38]:
w2v.similar_by_word("sond")

[('sonde', 0.9778403639793396),
 ('sony', 0.9761441946029663),
 ('sonde-sonde', 0.9716381430625916),
 ('sono', 0.97062748670578),
 ('sone', 0.9685573577880859),
 ('sonu', 0.9512014389038086),
 ("'sonde", 0.9447837471961975),
 ('son', 0.9033124446868896),
 ('onde', 0.8973442316055298),
 ('soni', 0.8945338726043701)]

In [39]:
fasttext_model.save(f"{PATH}/embedding/fasttext.18k.mkn.300.epoch5.uncased.bin")

In [40]:
w2v.save_word2vec_format(f"{PATH}/embedding/fasttext.18k.mkn.300.epoch5.uncased.vec.zip")

In [ ]:
!pip install gdown

In [ ]:
import gdown
import os

# URLs for the resources
url_vec_zip = 'https://drive.google.com/uc?id=1-3tOLyIY5fTUJs2L5CcyYhg82tL2W4TO'
url_npy = 'https://drive.google.com/uc?id=1WRV0nH5QjvJRyyaRKYMh19KS31t4NEqW'

# Output file paths
output_vec_zip = '/embedding/fasttext.18k.mkn.300.epoch5.uncased.vec.zip'
output_npy = '/embedding/fasttext.18k.mkn.300.epoch5.uncased.bin.wv.vectors_ngrams.npy'

# Create the embedding directory if it doesn't exist
os.makedirs('/embedding', exist_ok=True)

# Download files only if they do not already exist
if not os.path.exists(output_vec_zip):
    gdown.download(url_vec_zip, output_vec_zip, quiet=False)

if not os.path.exists(output_npy):
    gdown.download(url_npy, output_npy, quiet=False)

In [ ]:
cc = FastText.load("/content/fasttext.18k.mkn.300.epoch5.uncased.vec.zip")

In [48]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("/content/fasttext.18k.mkn.300.epoch5.uncased.vec.zip")